In [1]:
import torch  # PyTorch library for deep learning
import torch.nn as nn  # Module to create neural network layers
import torch.nn.functional as F  # Functional interface containing typical operations used for building neural networks like activation functions
import torch.optim as optim  # Optimizers for training neural networks
from torchvision import datasets, transforms  # Datasets and transformations utilities from torchvision
!pip install torchsummary  # Installing the torchsummary package for model summaries
from torchsummary import summary  # Importing the summary function from torchsummary for detailed model summaries


In [2]:
use_cuda = torch.cuda.is_available()  # Check if CUDA (GPU support) is available
device = torch.device("cuda" if use_cuda else "cpu")  # Select GPU if available, otherwise fall back to CPU
device  # Display the selected device

device(type='cuda')

In [3]:
batch_size = 1024  # Setting the batch size for training and testing

# Loading and transforming the MNIST training dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),  # Convert images to PyTorch tensors
                        transforms.Normalize((0.1307,), (0.3081,))  # Normalize images
                    ])),
    batch_size=batch_size, shuffle=True)  # DataLoader with specified batch size and shuffling

# Loading and transforming the MNIST testing dataset
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),  # Convert images to PyTorch tensors
                        transforms.Normalize((0.1307,), (0.3081,))  # Normalize images
                    ])),
    batch_size=batch_size, shuffle=True)  # DataLoader with specified batch size and shuffling


100%|██████████| 9912422/9912422 [00:00<00:00, 155808810.01it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 45797993.88it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 41298026.41it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 21190799.52it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



# Some Notes on our naive model

We are going to write a network based on what we have learnt so far.

The size of the input image is 28x28x1. We are going to add as many layers as required to reach RF = 32 "atleast".

In [ ]:
nout =

In [6]:
class FirstDNN(nn.Module): #the base class for all neural network modules in PyTorch.
  def __init__(self): #The initializer method defines the layers and components of the neural network.
    super(FirstDNN, self).__init__() #This line initializes the parent class (nn.Module), enabling the class to utilize all functionalities provided by PyTorch's nn.Module.
    # r_in:1, n_in:28, j_in:1, s:1, r_out:3, n_out:28, j_out:1
    self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
    # r_in: 3, n_in:28 , j_in:1 , s:1 , r_out:5 , n_out: 28, j_out:1
    self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
    # r_in:5 , n_in:28, j_in:1 , s:2 , r_out:6 , n_out:14 , j_out:2
    self.pool1 = nn.MaxPool2d(2, 2)
    # r_in:6 , n_in:14 , j_in:2 , s:1 , r_out:10 , n_out:14 , j_out:2
    self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
    # r_in:10 , n_in:14 , j_in:2 , s:1 , r_out:14 , n_out:14 , j_out:2
    self.conv4 = nn.Conv2d(128, 256, 3, padding = 1)
    # r_in:14 , n_in:14 , j_in:2 , s:2 , r_out: 16, n_out:7 , j_out:4
    self.pool2 = nn.MaxPool2d(2, 2)
    # r_in:16 , n_in:7 , j_in:4 , s:1 , r_out:24 , n_out: 5, j_out:4
    self.conv5 = nn.Conv2d(256, 512, 3)
    # r_in:24 , n_in:5 , j_in:4 , s:1 , r_out:32 , n_out:3 , j_out:4
    self.conv6 = nn.Conv2d(512, 1024, 3)
    # r_in:32 , n_in:3 , j_in:4 , s:1 , r_out:40 , n_out: 1, j_out:4
    self.conv7 = nn.Conv2d(1024, 10, 3)
# Correct values
# https://user-images.githubusercontent.com/498461/238034116-7db4cec0-7738-42df-8b67-afa971428d39.png
  def forward(self, x):
    x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
    x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
    x = F.relu(self.conv6(F.relu(self.conv5(x))))
    x = self.conv7(x)
    x = F.relu(x) # Activation function applied to the output of the last convolutional layer
    x = x.view(-1, 10)  # Flattening the output for the classification layer
    return F.log_softmax(x)


In [7]:
model = FirstDNN().to(device) #This operation ensures that the model will run on a GPU if one is available (and CUDA is enabled), otherwise, it will run on the CPU.

In [8]:
summary(model, input_size=(1, 28, 28)) #function then prints a detailed summary of the model, including the layers, their types, output shapes, and the number of parameters (both trainable and non-trainable). This is useful for understanding the architecture of the model, ensuring it is structured as intended, and estimating its computational complexity and memory usage.

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

<ipython-input-6-fd10e37ac998>:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [9]:
from tqdm import tqdm

def train(model, device, train_loader, optimizer, epoch):
    # Sets the model in training mode. This is necessary because certain modules like Dropout and BatchNorm
    # behave differently during training vs testing.
    model.train()

    # Wraps the train_loader with tqdm for a progress bar
    pbar = tqdm(train_loader)

    # Enumerates through all batches provided by train_loader
    for batch_idx, (data, target) in enumerate(pbar):
        # Sends the data and target values to the device (CPU/GPU)
        data, target = data.to(device), target.to(device)

        # Clears the gradients of all optimized variables
        optimizer.zero_grad()

        # Forward pass: compute the model output given data
        output = model(data)

        # Compute the loss value
        loss = F.nll_loss(output, target)

        # Backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()

        # Perform a single optimization step (parameter update)
        optimizer.step()

        # Update the progress bar with the current loss value and the batch index
        pbar.set_description(desc=f'loss={loss.item()} batch_id={batch_idx}')



def test(model, device, test_loader):
    # Sets the model in evaluation mode. This is necessary because certain modules like Dropout and BatchNorm
    # behave differently during training vs testing.
    model.eval()

    # Initializes the test loss and the number of correct predictions to 0
    test_loss = 0
    correct = 0

    # Disables gradient calculation to save memory and computations, since it's not needed during testing
    with torch.no_grad():
        # Enumerates through all batches provided by test_loader
        for data, target in test_loader:
            # Sends the data and target values to the device (CPU/GPU)
            data, target = data.to(device), target.to(device)

            # Forward pass: compute the model output given data
            output = model(data)

            # Accumulate the test loss by summing up losses of all batches
            test_loss += F.nll_loss(output, target, reduction='sum').item()

            # Get the index of the max log-probability as the prediction
            pred = output.argmax(dim=1, keepdim=True)

            # Counts the number of correct predictions
            correct += pred.eq(target.view_as(pred)).sum().item()

    # Compute the average loss
    test_loss /= len(test_loader.dataset)

    # Print the results for this epoch
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [10]:
# Initialize the Stochastic Gradient Descent (SGD) optimizer with specific parameters for the model.
# model.parameters() provides all the trainable parameters of the model to the optimizer.
# lr=0.01 sets the learning rate, which controls how much the weights of our model are adjusted with respect to the loss gradient.
# momentum=0.9 helps in smoothing out the updates by considering the past gradients, which can accelerate training and reduce oscillations.
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Loop through epochs for training and testing. Here, it's set to run for 1 epoch as range(1, 2) generates a sequence that starts at 1 and stops before 2.
for epoch in range(1, 2):
    # Call the train function to train the model for one epoch.
    # The function takes the model, device (CPU or GPU), training data loader, optimizer, and the current epoch index.
    train(model, device, train_loader, optimizer, epoch)

    # After training, call the test function to evaluate the model on the test dataset.
    # This helps in monitoring the model's performance and generalization ability.
    test(model, device, test_loader)


  0%|          | 0/59 [00:00<?, ?it/s]<ipython-input-6-fd10e37ac998>:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=2.177619457244873 batch_id=58: 100%|██████████| 59/59 [00:26<00:00,  2.21it/s]



Test set: Average loss: 2.1579, Accuracy: 3425/10000 (34%)



In [14]:
optimizer1 = optim.SGD(model.parameters(), lr=0.001, momentum=0.8)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer1, epoch)
    test(model, device, test_loader)

  0%|          | 0/59 [00:00<?, ?it/s]<ipython-input-6-fd10e37ac998>:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=2.302586078643799 batch_id=58: 100%|██████████| 59/59 [00:27<00:00,  2.18it/s]



Test set: Average loss: 2.3026, Accuracy: 980/10000 (10%)

